<a href="https://colab.research.google.com/github/samiravieira16-ui/projeto-dengue-letalidade/blob/main/Limpeza_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIMPEZA E FILTRAGEM DOS DADOS

# **Passo inical - Obtenção dos Dados**
---
Os dados foram obtidos pelo Sistema de Informação de Agravos de Notificação (Sinan) no link " https://dadosabertos.saude.gov.br/dataset/arboviroses-dengue ", todo o pacote de dados separado por ano está no formato CSV. Dessa forma, foi feito uma conversão para o formato PARQUET que é mais leve em tamanho.
Código utilizado:

In [ ]:
import pandas as pd
import os
import gc
import time
from google.colab import drive

print("Conectando ao Google Drive...")
# Força o remount para garantir que os atalhos atualizados foram lidos
drive.mount('/content/drive', force_remount=True)

# ====================================================================================
# CAMINHOS (Atualizados para MyDrive)
# ====================================================================================
# Pasta onde estão os arquivos brutos (CSV)
PASTA_ORIGEM = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Arquivos_Originais_Brutos.csv"

# Destino no seu Drive pessoal (onde os Parquet serão salvos)
PASTA_DESTINO = "/content/drive/MyDrive/Projeto Integrado (2º Semestre)/Dataset_Reduzido_Final"
os.makedirs(PASTA_DESTINO, exist_ok=True)

# Lista de arquivos para processar
arquivos_para_processar = [
    "DENGBR21.csv",
    "DENGBR22.csv",
    "DENGBR23.csv",
    "DENGBR24.csv",
    "DENGBR25.csv"
]

colunas_para_remover = [
    "UF_Residencia", "Municipio_Residencia", "Cod_Municipio_Residencia", "Pais_Residencia",
    "Ocupacao", "Sintoma_Febre", "Sintoma_DorCorpo", "Sintoma_DorCabeca", "Sintoma_Manchas",
    "Sintoma_Vomito", "Sintoma_Nausea", "Sintoma_DorCostas", "Sintoma_Conjuntivite",
    "Sintoma_Artrite", "Sintoma_DorArticulacao", "Sintoma_Petequias", "Sintoma_Leucopenia",
    "Sintoma_ProvaLaco", "Sintoma_DorRetroorbital", "DT_CHIK_S1", "DT_CHIK_S2", "DT_PRNT",
    "RES_CHIKS1", "RES_CHIKS2", "RESUL_PRNT", "Alarme_SangramentoMucosa", "Alarme_AumentoHematocrito",
    "Alarme_DorAbdominal", "Alarme_Letargia", "ALRM_HEPAT", "ALRM_LIQ", "DT_ALRM", "Gravidade_PulsoDebil",
    "Gravidade_PA_Convergente", "Gravidade_EnchimentoLento", "Gravidade_InsufRespiratoria",
    "Gravidade_Taquicardia", "Gravidade_ExtremidadesFrias", "GRAV_HIPOT", "GRAV_HEMAT",
    "GRAV_MELEN", "GRAV_METRO", "GRAV_SANG", "GRAV_AST", "GRAV_MIOC", "GRAV_CONSC", "GRAV_ORGAO",
    "DT_GRAV", "MANI_HEMOR", "EPISTAXE", "GENGIVO", "METRO", "PETEQUIAS", "HEMATURA", "SANGRAM",
    "LACO_N", "PLASMATICO", "EVIDENCIA", "PLAQ_MENOR", "CON_FHD", "COMPLICA", "TP_SISTEMA",
    "NDUPLIC_N", "CS_FLXRET", "FLXRECEBI", "MIGRADO_W", "DOENCA_TRA", "CLINC_CHIK",
    "Alarme_VomitoPersistente", "Alarme_Hipotensao", "Alarme_QuedaPlaquetas", "Data_Internacao",
    "DT_PCR", "DT_SORO", "DT_NS1"
]

codigos_dengue_positivo = ['10', '11', '12', 10, 11, 12, '10.0', '11.0', '12.0']

dict_renomear = {
    # Identificação e Geral
    'TP_NOT': 'Tipo_Notificacao', 'ID_AGRAVO': 'Codigo_Doenca', 'DT_NOTIFIC': 'Data_Notificacao',
    'NU_ANO': 'Ano_Notificacao', 'SG_UF_NOT': 'UF_Notificacao', 'UF': 'UF_Residencia',
    'MUNICIPIO': 'Municipio_Residencia', 'ID_MN_RESI': 'Cod_Municipio_Residencia', 'ID_PAIS': 'Pais_Residencia',
    'DT_SIN_PRI': 'Data_Inicio_Sintomas', 'NU_IDADE_N': 'Idade_Codificada',
    'CS_SEXO': 'Sexo', 'CS_RACA': 'Raca_Cor', 'CS_ESCOL_N': 'Escolaridade',
    'CS_GESTANT': 'Gestante', 'ID_OCUPA_N': 'Ocupacao',
    'CLASSI_FIN': 'Classificacao_Final', 'CRITERIO': 'Criterio_Confirmacao',
    'EVOLUCAO': 'Desfecho_Caso', 'DT_OBITO': 'Data_Obito',
    'DT_ENCERRA': 'Data_Encerramento', 'HOSPITALIZ': 'Houve_Hospitalizacao',
    'DT_INTERNA': 'Data_Internacao', 'DT_INVEST': 'Data_Investigacao',
    'SEM_PRI': 'Semana_Inicio_Sintomas', 'ANO_NASC': 'Ano_Nascimento',
    'SG_UF': 'UF_Residencia_Sigla', 'ID_RG_RESI': 'Regiao_Saude_Residencia',
    'TPAUTOCTO': 'Tipo_Autoctonia', 'COUFINF': 'UF_Infeccao',
    'COPAISINF': 'Pais_Infeccao', 'COMUNINF': 'Municipio_Infeccao',
    'DT_DIGITA': 'Data_Digitacao',
    # Sintomas
    'FEBRE': 'Sintoma_Febre', 'MIALGIA': 'Sintoma_Mialgia', 'CEFALEIA': 'Sintoma_Cefaleia',
    'EXANTEMA': 'Sintoma_Exantema', 'VOMITO': 'Sintoma_Vomito', 'NAUSEA': 'Sintoma_Nausea',
    'DOR_COSTAS': 'Sintoma_Dor_Costas', 'CONJUNTVIT': 'Sintoma_Conjuntivite',
    'ARTRITE': 'Sintoma_Artrite', 'ARTRALGIA': 'Sintoma_Artralgia',
    'PETEQUIA_N': 'Sintoma_Petequias', 'LEUCOPENIA': 'Sintoma_Leucopenia',
    'LACO': 'Sintoma_Laco', 'DOR_RETRO': 'Sintoma_Dor_Retroorbital',
    # Comorbidades
    'DIABETES': 'Comorbidade_Diabetes', 'HEMATOLOG': 'Comorbidade_Hematologica',
    'HEPATOPAT': 'Comorbidade_Hepatopatia', 'RENAL': 'Comorbidade_Renal',
    'HIPERTENSA': 'Comorbidade_Hipertensao', 'ACIDO_PEPT': 'Comorbidade_Acido_Peptica',
    'AUTO_IMUNE': 'Comorbidade_Autoimune',
    # Sinais de Alarme
    'ALRM_HIPOT': 'Alarme_Hipotensao', 'ALRM_PLAQ': 'Alarme_Plaquetopenia',
    'ALRM_VOM': 'Alarme_Vomitos_Persistentes', 'ALRM_SANG': 'Alarme_Sangramento_Mucosa',
    'ALRM_HEMAT': 'Alarme_Aumento_Hematocrito', 'ALRM_ABDOM': 'Alarme_Dor_Abdominal',
    'ALRM_LETAR': 'Alarme_Letargia',
    # Sinais de Gravidade
    'GRAV_PULSO': 'Gravidade_Pulso_Debil', 'GRAV_CONV': 'Gravidade_PA_Convergente',
    'GRAV_ENCH': 'Gravidade_Enchimento_Capilar_Lento', 'GRAV_INSUF': 'Gravidade_Insuficiencia_Respiratoria',
    'GRAV_TAQUI': 'Gravidade_Taquicardia', 'GRAV_EXTRE': 'Gravidade_Extremidades_Frias',
    # Exames
    'RESUL_SORO': 'Resultado_Sorologia', 'RESUL_NS1': 'Resultado_NS1',
    'DT_VIRAL': 'Data_Isolamento_Viral', 'RESUL_VI_N': 'Resultado_Isolamento_Viral',
    'RESUL_PCR_': 'Resultado_PCR', 'SOROTIPO': 'Sorotipo_Viral',
    'HISTOPA_N': 'Resultado_Histopatologia', 'IMUNOH_N': 'Resultado_Imunohistoquimica'
}


Conectando ao Google Drive...
Mounted at /content/drive


In [ ]:
# ====================================================================================
# PROCESSAMENTO E VALIDAÇÃO CONSOLIDADA
# ====================================================================================
import pandas as pd
import os
import time
import gc

print(f"\nProcurando a pasta de origem em: {PASTA_ORIGEM}")

if not os.path.exists(PASTA_ORIGEM):
    print(f"\n[ERRO] A pasta '{PASTA_ORIGEM}' não existe ou o Drive não foi montado corretamente.")
else:
    for nome_csv in arquivos_para_processar:
        caminho_csv = os.path.join(PASTA_ORIGEM, nome_csv)
        nome_parquet = os.path.splitext(nome_csv)[0] + "_Otimizado.parquet"
        caminho_parquet = os.path.join(PASTA_DESTINO, nome_parquet)

        print(f"\n--- PROCESSANDO: {nome_csv} ---")

        if not os.path.exists(caminho_csv):
            print(f"   [AVISO] Arquivo '{caminho_csv}' não encontrado. Pulando.")
            continue

        try:
            start = time.time()

            # Cálculo do tamanho original em GB
            tamanho_csv_gb = os.path.getsize(caminho_csv) / (1024**3)
            print(f"   [INFO] Tamanho do arquivo bruto (CSV): {tamanho_csv_gb:.4f} GB")

            # 1. Leitura Otimizada
            df = pd.read_csv(caminho_csv, sep=',', encoding='ISO-8859-1', dtype=str)

            # 2. Limpeza de Colunas
            colunas_para_dropar = [c for c in colunas_para_remover if c in df.columns]
            df.drop(columns=colunas_para_dropar, inplace=True)

            # 3. Filtro de Casos de Dengue
            if 'CLASSI_FIN' in df.columns:
                df = df[df['CLASSI_FIN'].isin(codigos_dengue_positivo)]

            # 4. Renomeação
            df.rename(columns=dict_renomear, inplace=True)

            # 5. Otimização de Memória (Categorias)
            for col in df.columns:
                if df[col].dtype == 'object' and (df[col].nunique() / len(df) < 0.5):
                    df[col] = df[col].astype('category')

            # 6. Salvar em Parquet
            df.to_parquet(caminho_parquet, index=False, engine='pyarrow', compression='zstd')

            # Cálculo do tamanho convertido em GB
            tamanho_parquet_gb = os.path.getsize(caminho_parquet) / (1024**3)
            tempo_proc = time.time() - start

            print(f"   [SUCESSO] Arquivo convertido: {tamanho_parquet_gb:.4f} GB")
            print(f"   [METRICA] Tempo: {tempo_proc:.1f}s | Redução de tamanho: {((tamanho_csv_gb - tamanho_parquet_gb)/tamanho_csv_gb)*100:.1f}%")

            # 7. Validação de Integridade
            print(f"   [VALIDAÇÃO] Verificando dados...")
            df_val = pd.read_parquet(caminho_parquet)
            print(f"      - Linhas finais: {len(df_val)}")
            del df_val

        except Exception as e:
            print(f"   [ERRO] Falha ao processar {nome_csv}: {e}")

        finally:
            if 'df' in locals(): del df
            gc.collect()

print("\n=============== PIPELINE CONCLUÍDO! ===============")


Procurando a pasta de origem em: /content/drive/MyDrive/Projeto Integrado (2º Semestre)/PROJETO II/Arquivos_Originais_Brutos.csv

--- PROCESSANDO: DENGBR21.csv ---
   [INFO] Tamanho do arquivo bruto (CSV): 0.2564 GB
   [SUCESSO] Arquivo convertido: 0.0073 GB
   [METRICA] Tempo: 33.3s | Redução de tamanho: 97.2%
   [VALIDAÇÃO] Verificando dados...
      - Linhas finais: 447884

--- PROCESSANDO: DENGBR22.csv ---
   [INFO] Tamanho do arquivo bruto (CSV): 0.3526 GB
   [SUCESSO] Arquivo convertido: 0.0218 GB
   [METRICA] Tempo: 55.0s | Redução de tamanho: 93.8%
   [VALIDAÇÃO] Verificando dados...
      - Linhas finais: 1243350

--- PROCESSANDO: DENGBR23.csv ---
   [INFO] Tamanho do arquivo bruto (CSV): 0.3924 GB
   [SUCESSO] Arquivo convertido: 0.0213 GB
   [METRICA] Tempo: 60.2s | Redução de tamanho: 94.6%
   [VALIDAÇÃO] Verificando dados...
      - Linhas finais: 1297582

--- PROCESSANDO: DENGBR24.csv ---
   [INFO] Tamanho do arquivo bruto (CSV): 1.6715 GB


# **Observação sobre o Ambiente de Execução:**
---
O código apresentado no Colab é uma demonstração do fluxo de ETL. Devido ao volume de dados (milhões de registros em arquivos como DENGBR24.csv) e ao teto de hardware da nuvem, o processamento completo foi executado em ambiente local (VS Code). O Colab exibe a transformação bem-sucedida de uma amostra representativa para validação da metodologia pelo professor.